-----import data----

In [11]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('mydm-3.db')

# 1.
bookings_df = pd.read_csv("Bookings.csv", index_col=0)
print("\nBookings Dataset ：", bookings_df.columns.tolist())

# 2.
payments_df = pd.read_csv("Payments.csv", index_col=0)
print("\nPayments Dataset ：", payments_df.columns.tolist())

# 3.
refunds_df = pd.read_csv("Refunds.csv", index_col=0)
print("\nRefunds Dataset ：", refunds_df.columns.tolist())

# 4.
reviews_df = pd.read_csv("Reviews.csv", index_col=0)
print("\nReviews Dataset ：", reviews_df.columns.tolist())

# 5.
user_activity_df = pd.read_csv("User_activity.csv", index_col=0)
print("\nUser Activity Dataset ：", user_activity_df.columns.tolist())

# 6.
users_df = pd.read_csv("Users.csv", index_col=0)
print("\nUsers Dataset ：", users_df.columns.tolist())

# 7.
cities_df = pd.read_csv("UK_cities.csv", index_col=0)
print("\nUK Cities Dataset ：", cities_df.columns.tolist())

# 8. house data
listings_df = pd.read_csv("Listings.csv", index_col=0)
print("\nListings Dataset ：", listings_df.columns.tolist())

# 9. lising amenities data
listing_amenities_df = pd.read_csv("Listing_amenities.csv", index_col=0)
print("\nListing Amenities Dataset ：", listing_amenities_df.columns.tolist())

# 10.
amenities_df = pd.read_csv("Amenities.csv", index_col=0)
print("\nAmenities Dataset ：", amenities_df.columns.tolist())


Bookings Dataset ： ['check_in_date', 'check_out_date', 'booking_status', 'booking_date', 'host_response_date', 'host_response_status', 'num_guests', 'guest_id', 'listing_id']

Payments Dataset ： ['payment_method', 'payment_status', 'payment_amount', 'payment_date', 'booking_id']

Refunds Dataset ： ['refund_type', 'refund_status', 'refund_amount', 'request_date', 'processed_date', 'booking_id', 'payment_id']

Reviews Dataset ： ['rating_score', 'review_text', 'review_date', 'response_text', 'response_date', 'booking_id', 'listing_id']

User Activity Dataset ： ['session_date', 'time_spent_page', 'device_type', 'conversion_status', 'clicks_per_sesh', 'user_id', 'booking_success', 'listing_clicked', 'booking_id']

Users Dataset ： ['email', 'first_name', 'last_name', 'phone_num', 'registration_date', 'gender', 'user_type', 'cities_id']

UK Cities Dataset ： ['cities_name']

Listings Dataset ： ['title', 'description', 'property_type', 'street', 'house_num', 'price_per_night', 'availability_st

Checking data validity

In [12]:
import sqlite3

conn = sqlite3.connect('mydm-3.db')

# Insert DataFrame data into SQLite database
cities_df.to_sql('UK_cities', conn, if_exists='append', index=False)
users_df.to_sql('Users', conn, if_exists='append', index=False)
listings_df.to_sql('Listings', conn, if_exists='append', index=False)
bookings_df.to_sql('Bookings', conn, if_exists='append', index=False)
payments_df.to_sql('Payments', conn, if_exists='append', index=False)
reviews_df.to_sql('Reviews', conn, if_exists='append', index=False)
refunds_df.to_sql('Refunds', conn, if_exists='append', index=False)
user_activity_df.to_sql('User_activity', conn, if_exists='append', index=False)
amenities_df.to_sql('Amenities', conn, if_exists='append', index=False)
listing_amenities_df.to_sql('Listing_amenities', conn, if_exists='append', index=False)


500

In [ ]:
cursor = conn.cursor()
cursor.execute("PRAGMA table_info(bookings);")
print(cursor.fetchall())


[(0, 'booking_id', 'INTEGER', 0, None, 1), (1, 'check_in_date', 'TEXT', 0, None, 0), (2, 'check_out_date', 'TEXT', 0, None, 0), (3, 'booking_status', 'TEXT', 0, None, 0), (4, 'booking_date', 'TEXT', 0, None, 0), (5, 'host_response_date', 'TEXT', 0, None, 0), (6, 'host_response_status', 'TEXT', 0, None, 0), (7, 'num_guests', 'INTEGER', 0, None, 0), (8, 'guest_id', 'INTEGER', 1, None, 0), (9, 'listing_id', 'INTEGER', 1, None, 0)]


In [ ]:
conn = sqlite3.connect("mydm-3.db")
cursor = conn.cursor()

cursor.execute("VACUUM;")
print("database has been fixed 。")



database has been fixed 。


In [3]:
import sqlite3
import pandas as pd


conn = sqlite3.connect('mydm-3.db')
cursor = conn.cursor()


cursor.execute("SELECT check_in_date FROM bookings LIMIT 5;")
sample_dates = cursor.fetchall()
print("date format check：", sample_dates)


def check_foreign_key(child_table, child_col, parent_table, parent_col):
    cursor.execute(f"""
    SELECT COUNT(*) FROM {child_table}
    WHERE {child_col} NOT IN (SELECT {parent_col} FROM {parent_table})
    """)
    invalid = cursor.fetchone()[0]
    print(f"{child_table}.{child_col} invalid number is：{invalid}")

check_foreign_key("listings", "cities_id", "UK_cities", "cities_id")
check_foreign_key("bookings", "listing_id", "listings", "listing_id")
check_foreign_key("payments", "booking_id", "bookings", "booking_id")

date format check： [('2025-01-04',), ('2024-06-11',), ('2025-03-02',), ('2024-12-03',), ('2024-07-24',)]
listings.cities_id invalid number is：0
bookings.listing_id invalid number is：0
payments.booking_id invalid number is：0



# ---- sales analysis by city ----

Confirmed booking distribution（rank by NET）

In [14]:
import pandas as pd

def analyze_sales_by_city():

    query = """
    WITH paid_payments AS (
        -- only for number of paid
        SELECT
            booking_id,
            SUM(payment_amount) AS total_payment
        FROM payments
        WHERE payment_status = 'paid' -- Only count successful payment records
        GROUP BY booking_id
    ),
    ---Calculate the total refund amount for each order
    refund_agg AS (
        SELECT
            booking_id,
            SUM(refund_amount) AS total_refund
        FROM refunds
        GROUP BY booking_id
    )
    SELECT
        c.cities_name AS city,
        COUNT(DISTINCT b.booking_id) AS confirmed_bookings, -- Confirmed Orders
        COALESCE(SUM(p.total_payment), 0) - COALESCE(SUM(r.total_refund), 0) AS net_revenue,
        (COALESCE(SUM(p.total_payment), 0) - COALESCE(SUM(r.total_refund), 0))
          / NULLIF(COUNT(DISTINCT b.booking_id), 0) AS avg_booking_value
    FROM bookings b
    JOIN listings l ON b.listing_id = l.listing_id
    JOIN UK_cities c ON l.cities_id = c.cities_id
    LEFT JOIN paid_payments p ON b.booking_id = p.booking_id  -- connect to paid
    LEFT JOIN refund_agg r ON b.booking_id = r.booking_id     -- connect to refund
    WHERE p.booking_id IS NOT NULL  -- only payment has been confirmed
    GROUP BY city
    ORDER BY net_revenue DESC
    """
    cursor.execute(query)
    df = pd.DataFrame(
        cursor.fetchall(),
        columns=["city", "confirmed_bookings", "net_revenue", "avg_booking_value"]
    )
    return df

def analyze_city_performance():
    """

    """
    query = """
    WITH all_payments AS (
        -- （paid + pending）
        SELECT
            booking_id,
            SUM(payment_amount) AS total_payment
        FROM payments
        GROUP BY booking_id
    )
    SELECT
        c.cities_name AS city,
        COUNT(DISTINCT b.booking_id) AS total_bookings,  -- total bookings
        COALESCE(SUM(p.total_payment), 0) AS total_revenue  -- total revenue）
    FROM bookings b
    JOIN listings l ON b.listing_id = l.listing_id
    JOIN UK_cities c ON l.cities_id = c.cities_id
    LEFT JOIN all_payments p ON b.booking_id = p.booking_id
    GROUP BY c.cities_name
    ORDER BY total_revenue DESC
    """
    cursor.execute(query)
    df = pd.DataFrame(
        cursor.fetchall(),
        columns=["city", "total_bookings", "total_revenue"]
    )
    return df


sales_by_city = analyze_sales_by_city()
city_perf = analyze_city_performance()

merged_df = pd.merge(
    city_perf[["city", "total_bookings", "total_revenue"]],
    sales_by_city[["city", "confirmed_bookings", "net_revenue", "avg_booking_value"]],
    on="city",
    how="outer"  #
).fillna(0)

merged_df["loss"] = merged_df["total_revenue"] - merged_df["net_revenue"]

merged_df = merged_df[[
    "city",
    "total_bookings",
    "confirmed_bookings",
    "avg_booking_value",
    "total_revenue",
    "net_revenue",
    "loss"
]]

merged_df = merged_df.sort_values(by="net_revenue", ascending=False)

print("=== Booking Distribution by City ===")
print(merged_df.head(10))

=== Booking Distribution by City ===
                  city  total_bookings  confirmed_bookings  avg_booking_value  \
13  Kingston upon Hull              75                  19        3603.618447   
12             Glasgow              69                  11        4002.902966   
14               Leeds              66                  14        3117.493314   
24           Sheffield              39                   9        3890.997676   
3             Bradford              57                  11        2826.000000   
5            Cambridge              48                  10        3043.906511   
2           Birmingham              60                  12        2536.000000   
16           Liverpool              72                  15        1987.538235   
27             Swansea              69                  14        2082.184308   
11              Exeter              30                   9        2468.000000   

    total_revenue   net_revenue          loss  
13         102600  6846

confirmed booking distribution by city
（ranked by confirmed bookings）

In [ ]:
import pandas as pd

def analyze_sales_by_city():
    """

    """
    query = """
    WITH paid_payments AS (
        -- only for number of paid
        SELECT
            booking_id,
            SUM(payment_amount) AS total_payment
        FROM payments
        WHERE payment_status = 'paid'
        GROUP BY booking_id
    ),
    refund_agg AS (
        SELECT
            booking_id,
            SUM(refund_amount) AS total_refund
        FROM refunds
        GROUP BY booking_id
    )
    SELECT
        c.cities_name AS city,
        COUNT(DISTINCT b.booking_id) AS confirmed_bookings,
        COALESCE(SUM(p.total_payment), 0) - COALESCE(SUM(r.total_refund), 0) AS net_revenue,
        (COALESCE(SUM(p.total_payment), 0) - COALESCE(SUM(r.total_refund), 0))
          / NULLIF(COUNT(DISTINCT b.booking_id), 0) AS avg_booking_value
    FROM bookings b
    JOIN listings l ON b.listing_id = l.listing_id
    JOIN UK_cities c ON l.cities_id = c.cities_id
    LEFT JOIN paid_payments p ON b.booking_id = p.booking_id  -- connect to paid
    LEFT JOIN refund_agg r ON b.booking_id = r.booking_id     -- connect to refund
    WHERE p.booking_id IS NOT NULL  -- only payment has been confirmed
    GROUP BY city
    ORDER BY net_revenue DESC
    """
    cursor.execute(query)
    df = pd.DataFrame(
        cursor.fetchall(),
        columns=["city", "confirmed_bookings", "net_revenue", "avg_booking_value"]
    )
    return df

def analyze_city_performance():
    """

    """
    query = """
    WITH all_payments AS (
        -- （paid + pending）
        SELECT
            booking_id,
            SUM(payment_amount) AS total_payment
        FROM payments
        GROUP BY booking_id
    )
    SELECT
        c.cities_name AS city,
        COUNT(DISTINCT b.booking_id) AS total_bookings,  -- total bookings
        COALESCE(SUM(p.total_payment), 0) AS total_revenue  -- total revenue）
    FROM bookings b
    JOIN listings l ON b.listing_id = l.listing_id
    JOIN UK_cities c ON l.cities_id = c.cities_id
    LEFT JOIN all_payments p ON b.booking_id = p.booking_id
    GROUP BY c.cities_name
    ORDER BY total_revenue DESC
    """
    cursor.execute(query)
    df = pd.DataFrame(
        cursor.fetchall(),
        columns=["city", "total_bookings", "total_revenue"]
    )
    return df


sales_by_city = analyze_sales_by_city()
city_perf = analyze_city_performance()

merged_df = pd.merge(
    city_perf[["city", "total_bookings", "total_revenue"]],
    sales_by_city[["city", "confirmed_bookings", "net_revenue", "avg_booking_value"]],
    on="city",
    how="outer"  #
).fillna(0)

merged_df["loss"] = merged_df["total_revenue"] - merged_df["net_revenue"]

merged_df = merged_df[[
    "city",
    "total_bookings",
    "confirmed_bookings",
    "avg_booking_value",
    "total_revenue",
    "net_revenue",
    "loss"
]]

merged_df = merged_df.sort_values(by="confirmed_bookings", ascending=False)

print("=== Booking Distribution by City ===")
print(merged_df.head(10))

=== Booking Distribution by City ===
                  city  total_bookings  confirmed_bookings  avg_booking_value  \
13  Kingston upon Hull              50                  19        2402.412298   
6              Cardiff              60                  16         828.520569   
16           Liverpool              48                  15        1325.025490   
14               Leeds              44                  14        2078.328876   
27             Swansea              46                  14        1388.122872   
23             Reading              38                  13        1107.883406   
2           Birmingham              40                  12        1690.000000   
12             Glasgow              46                  11        2668.601977   
4              Bristol              34                  11        1289.762356   
3             Bradford              38                  11        1884.000000   

    total_revenue   net_revenue          loss  
13          68400  4564

The monthly regional distribution is very uniform,
with no much booking in each region each month,
so this trend may not good used for analysised.

#---Monthly regional booking trends---

In [ ]:
def analyze_monthly_trends():
    query = """
    SELECT
        strftime('%Y-%m', b.check_in_date) AS month,
        c.cities_name AS city,
        COUNT(b.booking_id) AS booking_count,
        SUM(p.payment_amount) AS monthly_revenue
    FROM bookings b
    JOIN listings l ON b.listing_id = l.listing_id
    JOIN UK_cities c ON l.cities_id = c.cities_id
    JOIN payments p ON b.booking_id = p.booking_id
    GROUP BY month, city
    ORDER BY month, monthly_revenue DESC
    """
    cursor.execute(query)
    df = pd.DataFrame(cursor.fetchall(), columns=["month", "city", "bookings", "revenue"])
    return df


monthly_trends = analyze_monthly_trends()
pivot_table = monthly_trends.pivot(index="month", columns="city", values="bookings").fillna(0)
print("\n=== Monthly regional booking trends===")
print(pivot_table.head())


=== Monthly regional booking trends===
city     Aberdeen  Belfast  Birmingham  Bradford  Bristol  Cambridge  Cardiff  \
month                                                                           
2024-03       6.0      0.0         0.0       0.0      0.0        0.0      2.0   
2024-04       2.0      2.0         0.0       4.0      2.0        4.0      6.0   
2024-05       4.0      0.0         4.0       4.0      0.0        0.0      8.0   
2024-06       4.0      0.0         4.0       0.0      2.0        0.0      6.0   
2024-07       4.0      0.0         2.0       4.0      2.0        6.0      6.0   

city     Coventry  Derby  Dundee  ...  Nottingham  Plymouth  Portsmouth  \
month                             ...                                     
2024-03       0.0    4.0     0.0  ...         0.0       0.0         0.0   
2024-04       2.0    0.0     2.0  ...         2.0       2.0         6.0   
2024-05       4.0    6.0     2.0  ...         0.0       2.0         0.0   
2024-06       2.0

# -- monthly trend--


In [ ]:
def analyze_monthly_trends():
    query = """
    -- Add EXPLAIN QUERY PLAN to analyze execution path
    EXPLAIN QUERY PLAN
    SELECT
        strftime('%Y-%m', b.check_in_date) AS month,
        c.cities_name AS city,
        COUNT(b.booking_id) AS booking_count,
        SUM(p.payment_amount) AS total_revenue
    FROM bookings b
    JOIN listings l ON b.listing_id = l.listing_id
    JOIN UK_cities c ON l.cities_id = c.cities_id
    JOIN payments p ON b.booking_id = p.booking_id
    GROUP BY month, city
    ORDER BY month, city
    """


    # Actually executing the query
    cursor.execute(query.replace("EXPLAIN QUERY PLAN", ""))
    df = pd.DataFrame(cursor.fetchall(), columns=["month", "city", "bookings", "revenue"])
    return df


monthly_trends = analyze_monthly_trends()
print("\n=== monthly trend ===")
print(monthly_trends.head(50))


=== monthly trend ===
      month                 city  bookings  revenue
0   2024-03             Aberdeen         6     3150
1   2024-03              Cardiff         2      280
2   2024-03                Derby         4     1710
3   2024-03            Edinburgh         4     2690
4   2024-03               Exeter         2      860
5   2024-03              Glasgow         2      540
6   2024-03   Kingston upon Hull         4     1290
7   2024-03                Leeds         2     5360
8   2024-03           Manchester         2      560
9   2024-03  Newcastle upon Tyne         2     2200
10  2024-03              Reading         2      280
11  2024-03              Swansea         2      400
12  2024-03        Wolverhampton         2      810
13  2024-03                 York         2      270
14  2024-04             Aberdeen         2      580
15  2024-04              Belfast         2     1400
16  2024-04             Bradford         4     2430
17  2024-04              Bristol         

#popularity of property type

> 添加区块引用符号



ranked by NET revenue

In [7]:
def analyze_property_performance():
    query = """
    SELECT
        l.property_type,
        COUNT(DISTINCT CASE WHEN p.payment_status = 'paid' THEN b.booking_id END) AS paid_bookings,
        SUM(p.payment_amount) AS total_revenue,
        SUM(CASE WHEN p.payment_status = 'paid' THEN p.payment_amount ELSE 0 END) AS paid_revenue,
        AVG(l.price_per_night) AS avg_price,
        (COUNT(DISTINCT CASE WHEN p.payment_status = 'paid' THEN b.booking_id END) * 1.0
          / (SELECT COUNT(DISTINCT booking_id) FROM payments WHERE payment_status = 'paid')) AS market_share
    FROM bookings b
    JOIN listings l ON b.listing_id = l.listing_id
    JOIN payments p ON b.booking_id = p.booking_id
    GROUP BY l.property_type
    ORDER BY total_revenue DESC
    """
    cursor.execute(query)
    df = pd.DataFrame(
        cursor.fetchall(),
        columns=["property_type", "paid_bookings", "total_revenue", "NET_revenue", "avg_price", "market_share"]
    )
    return df

property_perf = analyze_property_performance()
print("\n=== property type analysis===")
print(property_perf)


=== property type analysis===
  property_type  paid_bookings  total_revenue  NET_revenue   avg_price  \
0         Villa             64         325880       215200  264.702970   
1         House             44         149420        88310  199.709302   
2     Apartment             60          87510        49200  139.957265   
3       Cottage             43          76390        34290  148.315217   
4        Studio             54          74070        35400  139.038462   

   market_share  
0      0.241509  
1      0.166038  
2      0.226415  
3      0.162264  
4      0.203774  


ranked by booking


In [6]:
def analyze_property_performance():
    query = """
    SELECT
        l.property_type,
        COUNT(DISTINCT CASE WHEN p.payment_status = 'paid' THEN b.booking_id END) AS bookings,
        SUM(p.payment_amount) AS total_revenue,
        SUM(CASE WHEN p.payment_status = 'paid' THEN p.payment_amount ELSE 0 END) AS paid_revenue,
        AVG(l.price_per_night) AS avg_price,
        (COUNT(DISTINCT CASE WHEN p.payment_status = 'paid' THEN b.booking_id END) * 1.0
          / (SELECT COUNT(DISTINCT booking_id) FROM payments WHERE payment_status = 'paid')) AS market_share
    FROM bookings b
    JOIN listings l ON b.listing_id = l.listing_id
    JOIN payments p ON b.booking_id = p.booking_id
    GROUP BY l.property_type
    ORDER BY bookings DESC
    """
    cursor.execute(query)
    df = pd.DataFrame(
        cursor.fetchall(),
        columns=["property_type", "bookings", "total_revenue", "NET_revenue", "avg_price", "market_share"]
    )
    return df

property_perf = analyze_property_performance()
print("\n=== property type analysis）===")
print(property_perf)


=== property type analysis）===
  property_type  bookings  total_revenue  NET_revenue   avg_price  \
0         Villa        64         325880       215200  264.702970   
1     Apartment        60          87510        49200  139.957265   
2        Studio        54          74070        35400  139.038462   
3         House        44         149420        88310  199.709302   
4       Cottage        43          76390        34290  148.315217   

   market_share  
0      0.241509  
1      0.226415  
2      0.203774  
3      0.166038  
4      0.162264  


# Popularity of different property types in different cities

In [9]:
def analyze_property_performance():
    query = """
    SELECT
        l.property_type,

        COUNT(DISTINCT CASE WHEN p.payment_status = 'paid' THEN b.booking_id END) AS paid_bookings,
        SUM(CASE WHEN p.payment_status = 'paid' THEN p.payment_amount ELSE 0 END) AS paid_revenue,
        SUM(CASE WHEN p.payment_status IN ('paid', 'pending') THEN p.payment_amount ELSE 0 END) AS total_revenue_all,
        AVG(l.price_per_night) AS avg_price,
        (COUNT(DISTINCT CASE WHEN p.payment_status = 'paid' THEN b.booking_id END) * 1.0
          / (SELECT COUNT(DISTINCT booking_id) FROM payments WHERE payment_status = 'paid')) AS market_share
    FROM bookings b
    JOIN listings l ON b.listing_id = l.listing_id
    JOIN payments p ON b.booking_id = p.booking_id
    GROUP BY l.property_type
    ORDER BY paid_bookings DESC
    """
    cursor.execute(query)
    df = pd.DataFrame(
        cursor.fetchall(),
        columns=["property_type", "paid_bookings", "NET_revenue", "total_revenue_", "avg_price", "market_share"]
    )
    return df

property_perf = analyze_property_performance()
print("\n===property type analysis）===")
print(property_perf)


===property type analysis）===
  property_type  paid_bookings  NET_revenue  total_revenue_   avg_price  \
0         Villa             64       215200          314770  264.702970   
1     Apartment             60        49200           77270  139.957265   
2        Studio             54        35400           64010  139.038462   
3         House             44        88310          127980  199.709302   
4       Cottage             43        34290           67510  148.315217   

   market_share  
0      0.241509  
1      0.226415  
2      0.203774  
3      0.166038  
4      0.162264  


# --Premium rate and Demand-Supply Ratios--

Premium rate

In [ ]:
def analyze_premium_properties():
    query = """
    SELECT
        c.cities_name AS city,
        l.property_type,
        AVG(p.payment_amount) AS avg_revenue_per_booking,
        -- Calculate the premium index (the proportion of housing prices above the city average)）
        (AVG(l.price_per_night) / city_avg.avg_price - 1) * 100 AS premium_rate
    FROM bookings b
    JOIN listings l ON b.listing_id = l.listing_id
    JOIN UK_cities c ON l.cities_id = c.cities_id
    JOIN payments p ON b.booking_id = p.booking_id
    JOIN (
        SELECT cities_id, AVG(price_per_night) AS avg_price
        FROM listings
        GROUP BY cities_id
    ) city_avg ON l.cities_id = city_avg.cities_id
    GROUP BY city, property_type
    HAVING premium_rate > 20  -- Filter rooms with a premium of more than 20%
    ORDER BY premium_rate DESC
    """
    cursor.execute(query)
    df = pd.DataFrame(cursor.fetchall(), columns=["city", "property_type", "avg_revenue", "premium_rate"])
    return df


premium_properties = analyze_premium_properties()
print("\n=== High premium room types distribution ===")
print(premium_properties)


=== High premium room types distribution ===
                   city property_type  avg_revenue  premium_rate
0             Sheffield         Villa  4550.000000    100.883002
1              Aberdeen         Villa  2600.000000     86.602871
2                London         Villa   903.333333     82.608696
3            Manchester         Villa  1060.000000     79.840319
4              Bradford         Villa  3300.000000     78.915663
5                 Leeds         Villa  2240.625000     75.902669
6               Reading         House  2575.000000     71.406959
7               Swansea         Villa  2540.000000     70.085470
8        Stoke-on-Trent         Villa  1665.000000     65.531062
9                 Derby         House   632.500000     61.923688
10            Liverpool         Villa  1430.000000     61.126761
11               Exeter         House  1020.000000     55.015198
12           Portsmouth         Villa   620.000000     52.948479
13                 York         Villa  2340.

Demand_supply_ratio


In [5]:
def analyze_demand_supply_ratio():
    query = """
    SELECT
        c.cities_name AS city,
        l.property_type,
        COUNT(CASE WHEN b.booking_status = 'confirmed' THEN b.booking_id END) AS demand,
        COUNT(l.listing_id) AS supply,
        (COUNT(CASE WHEN b.booking_status = 'confirmed' THEN b.booking_id END) * 1.0 / NULLIF(COUNT(l.listing_id), 0)) AS demand_supply_ratio
    FROM listings l
    LEFT JOIN bookings b ON l.listing_id = b.listing_id
    JOIN UK_cities c ON l.cities_id = c.cities_id
    GROUP BY city, property_type
    ORDER BY demand_supply_ratio DESC;
    """
    cursor.execute(query)
    df = pd.DataFrame(cursor.fetchall(), columns=["city", "property_type", "demand", "supply", "demand_supply_ratio"])
    return df


demand_supply_data = analyze_demand_supply_ratio()
print("\n=== Demand-Supply Ratio Analysis ===")
print(demand_supply_data)
print("\n=== TOP 10 Demand-Supply Ratio Analysis ===")
print(demand_supply_data.head(10))



=== Demand-Supply Ratio Analysis ===
               city property_type  demand  supply  demand_supply_ratio
0        Portsmouth     Apartment       6       7             0.857143
1          Coventry     Apartment       4       5             0.800000
2            Dundee         Villa       4       5             0.800000
3         Edinburgh     Apartment       4       5             0.800000
4           Cardiff         Villa      10      13             0.769231
..              ...           ...     ...     ...                  ...
137  Stoke-on-Trent        Studio       0       7             0.000000
138   Wolverhampton       Cottage       0      10             0.000000
139   Wolverhampton         Villa       0       8             0.000000
140            York       Cottage       0       5             0.000000
141            York         House       0       9             0.000000

[142 rows x 5 columns]

=== TOP 10 Demand-Supply Ratio Analysis ===
         city property_type  demand  supp

Rating score by property type calculation

In [ ]:
def calculate_property_type_ratings():
    query = """
    SELECT
        l.property_type,
        ROUND(AVG(r.rating_score), 2) AS avg_rating,
        COUNT(r.rating_score) AS total_reviews
    FROM bookings b
    JOIN listings l ON b.listing_id = l.listing_id
    JOIN payments p ON b.booking_id = p.booking_id
    LEFT JOIN reviews r ON b.booking_id = r.booking_id
    WHERE p.payment_status = 'paid'
    GROUP BY l.property_type
    ORDER BY avg_rating DESC
    """
    cursor.execute(query)
    results = cursor.fetchall()
    df = pd.DataFrame(results, columns=["property_type", "avg_rating", "total_reviews"])
    return df


try:
    ratings_df = calculate_property_type_ratings()
    print("=== Property type avg score ===")
    print(ratings_df.head())
except Exception as e:
    print("errors:", e)


=== Property type avg score ===
  property_type  avg_rating  total_reviews
0       Cottage        3.80            204
1         House        3.70            244
2         Villa        3.69            288
3     Apartment        3.67            268
4        Studio        3.62            276
